In [1]:
import yfinance as yf
import pandas as pd
import pandas_ta as ta

def get_data(symbol: str):
    data = yf.download(tickers=symbol, period='300d', interval='1d')
    data.reset_index(inplace=True, drop=True)
    return data
# Get the data
data = get_data('BTC-USD')

[*********************100%%**********************]  1 of 1 completed


In [2]:
# Calculate Bollinger Bands using pandas_ta
data.ta.bbands(length=10, std=1.5, append=True)

# Add the upper and lower bands to the DataFrame
data['Upper Band'] = data['BBU_10_1.5']
data['Lower Band'] = data['BBL_10_1.5']

def calculate_sma(data, length: int):
    return ta.sma(data['Close'], length)

# Calculate the moving average
data['SMA'] = calculate_sma(data, 20)
data.dropna(inplace=True)

In [3]:
data

,Open,High,Low,Close,Adj Close,Volume,BBL_10_1.5,BBM_10_1.5,BBU_10_1.5,BBB_10_1.5,BBP_10_1.5,Upper Band,Lower Band,SMA
19,26188.691406,26220.201172,25846.087891,26124.140625,26124.140625,13371557893,25452.048449,27710.348633,29968.648816,16.299327,0.148805,29968.648816,25452.048449,28496.353516
20,26130.748047,26135.507812,25520.728516,26031.656250,26031.656250,14503820706,25176.077776,27371.917773,29567.757770,16.044473,0.194818,29567.757770,25176.077776,28340.338379
21,26040.474609,26786.898438,25804.998047,26431.640625,26431.640625,16985265785,25082.783966,27086.790430,29090.796893,14.796928,0.336540,29090.796893,25082.783966,28202.986426
22,26431.519531,26554.910156,25914.925781,26162.373047,26162.373047,12871532023,25101.320087,26762.183398,28423.046709,12.412017,0.319428,28423.046709,25101.320087,28057.400488
23,26163.679688,26248.103516,25786.812500,26047.667969,26047.667969,12406045118,25288.406102,26449.915430,27611.424758,8.782707,0.326843,27611.424758,25288.406102,27907.677539
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
295,69121.304688,70041.273438,66356.953125,67929.562500,67929.562500,41895680979,63270.891328,67195.953516,71121.015703,11.682436,0.593452,71121.015703,63270.891328,64777.657031
296,67928.132812,69220.296875,66622.671875,68526.101562,68526.101562,29197308153,65146.486413,67893.284766,70640.083118,8.091517,0.615192,70640.083118,65146.486413,65009.388477
297,68526.921875,69579.320312,68515.820312,69265.945312,69265.945312,15473071741,65515.102083,68193.130078,70871.158073,7.854246,0.700299,70871.158073,65515.102083,65271.129102
298,69264.289062,69506.226562,68183.890625,68518.093750,68518.093750,15628433737,66290.381323,68521.781250,70753.181177,6.512965,0.499174,70753.181177,66290.381323,65538.936328


In [4]:
def check_candles(data, backcandles, ma_column):
    categories = [0 for _ in range(backcandles)]
    for i in range(backcandles, len(data)):
        if all(data['Close'][i-backcandles:i] > data[ma_column][i-backcandles:i]):
            categories.append(2)  # Uptrend
        elif all(data['Close'][i-backcandles:i] < data[ma_column][i-backcandles:i]):
            categories.append(1)  # Downtrend
        else:
            categories.append(0)  # No trend
    return categories

# Apply the function to the DataFrame
data['Trend'] = check_candles(data, 7, 'SMA')

In [5]:
data

,Open,High,Low,Close,Adj Close,Volume,BBL_10_1.5,BBM_10_1.5,BBU_10_1.5,BBB_10_1.5,BBP_10_1.5,Upper Band,Lower Band,SMA,Trend
19,26188.691406,26220.201172,25846.087891,26124.140625,26124.140625,13371557893,25452.048449,27710.348633,29968.648816,16.299327,0.148805,29968.648816,25452.048449,28496.353516,0
20,26130.748047,26135.507812,25520.728516,26031.656250,26031.656250,14503820706,25176.077776,27371.917773,29567.757770,16.044473,0.194818,29567.757770,25176.077776,28340.338379,0
21,26040.474609,26786.898438,25804.998047,26431.640625,26431.640625,16985265785,25082.783966,27086.790430,29090.796893,14.796928,0.336540,29090.796893,25082.783966,28202.986426,0
22,26431.519531,26554.910156,25914.925781,26162.373047,26162.373047,12871532023,25101.320087,26762.183398,28423.046709,12.412017,0.319428,28423.046709,25101.320087,28057.400488,0
23,26163.679688,26248.103516,25786.812500,26047.667969,26047.667969,12406045118,25288.406102,26449.915430,27611.424758,8.782707,0.326843,27611.424758,25288.406102,27907.677539,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
295,69121.304688,70041.273438,66356.953125,67929.562500,67929.562500,41895680979,63270.891328,67195.953516,71121.015703,11.682436,0.593452,71121.015703,63270.891328,64777.657031,2
296,67928.132812,69220.296875,66622.671875,68526.101562,68526.101562,29197308153,65146.486413,67893.284766,70640.083118,8.091517,0.615192,70640.083118,65146.486413,65009.388477,2
297,68526.921875,69579.320312,68515.820312,69265.945312,69265.945312,15473071741,65515.102083,68193.130078,70871.158073,7.854246,0.700299,70871.158073,65515.102083,65271.129102,2
298,69264.289062,69506.226562,68183.890625,68518.093750,68518.093750,15628433737,66290.381323,68521.781250,70753.181177,6.512965,0.499174,70753.181177,66290.381323,65538.936328,2


## 1 - Entry Based on Bollinger Bands - Candles Crossing

In [6]:
# Check conditions and assign entry values
data['entry'] = 0

# Condition for entry category 2 (buy entry)
buy_entry_condition = (data['Trend'] == 2) & ((data['Open'] < data['Lower Band']) & (data['Close'] > data['Lower Band']))
data.loc[buy_entry_condition, 'entry'] = 2

# Condition for entry category 1 (sell entry)
sell_entry_condition = (data['Trend'] == 1) & ((data['Open'] > data['Upper Band']) & (data['Close'] < data['Upper Band']))
data.loc[sell_entry_condition, 'entry'] = 1

In [7]:
data[data['entry']!=0]

,Open,High,Low,Close,Adj Close,Volume,BBL_10_1.5,BBM_10_1.5,BBU_10_1.5,BBB_10_1.5,BBP_10_1.5,Upper Band,Lower Band,SMA,Trend,entry
133,41468.464844,43429.781250,40676.867188,42890.742188,42890.742188,26797884674,41474.546768,43035.693750,44596.840732,7.255126,0.453575,44596.840732,41474.546768,40607.654883,2,2
206,50736.371094,51684.195312,50585.445312,51571.101562,51571.101562,15174077879,51083.919544,51739.552344,52395.185144,2.534358,0.371536,52395.185144,51083.919544,49374.920312,2,2


In [8]:
import plotly.graph_objects as go

dfpl = data[:]
fig = go.Figure(data=[go.Candlestick(x=dfpl.index,
                open=dfpl['Open'],
                high=dfpl['High'],
                low=dfpl['Low'],
                close=dfpl['Close'])])

# Add the moving averages to the plot
fig.add_trace(go.Scatter(x=dfpl.index, y=dfpl['SMA'], mode='lines', name='SMA', line=dict(color='red')))
fig.add_trace(go.Scatter(x=dfpl.index, y=dfpl['Lower Band'], mode='lines', name='Lower Band', line=dict(color='blue')))
fig.add_trace(go.Scatter(x=dfpl.index, y=dfpl['Upper Band'], mode='lines', name='Upper Band', line=dict(color='blue')))

fig.update_layout(xaxis_rangeslider_visible=False)
fig.show()

## 2 - Entry based on RSI and Bollinger Bands

In [9]:
def add_rsi_column(data):
    # Calculate RSI with a period of 14
    data['RSI'] = ta.rsi(data['Close'])
    return data

data = add_rsi_column(data)

In [10]:
def rsi_signal(data):
    data['RSI Signal'] = 0  # Initialize the signal column with 0

    # Set the signal category to 2 when the price is below the lower Bollinger Band and RSI is below 30
    data.loc[(data['Close'] < data['Lower Band']) & (data['RSI'] < 55), 'RSI Signal'] = 2

    # Set the signal category to 1 when the price is above the upper Bollinger Band and RSI is above 70
    data.loc[(data['Close'] > data['Upper Band']) & (data['RSI'] > 45), 'RSI Signal'] = 1

    return data

In [11]:
data = rsi_signal(data)

In [12]:
data[data["RSI Signal"]!=0]

,Open,High,Low,Close,Adj Close,Volume,BBL_10_1.5,BBM_10_1.5,BBU_10_1.5,BBB_10_1.5,BBP_10_1.5,Upper Band,Lower Band,SMA,Trend,entry,RSI,RSI Signal
40,25831.714844,25883.947266,24930.296875,25162.654297,25162.654297,14600006467,25438.818902,25822.040820,26205.262739,2.968177,-0.360319,26205.262739,25438.818902,26091.162598,0,0,36.202671,2
43,26228.277344,26774.623047,26171.451172,26539.673828,26539.673828,13811359124,25393.491876,25917.096875,26440.701874,4.040615,1.094510,26440.701874,25393.491876,26089.145605,0,0,56.681151,1
44,26533.818359,26840.498047,26240.701172,26608.693359,26608.693359,11479735788,25398.221550,25999.967969,26601.714387,4.628824,1.005799,26601.714387,25398.221550,26119.157129,0,0,57.488434,1
48,26760.851562,27488.763672,26681.605469,27211.117188,27211.117188,13807690550,25499.706233,26327.278320,27154.850407,6.286803,1.033995,27154.850407,25499.706233,26111.525391,0,0,64.552782,1
53,26579.373047,26716.058594,26221.050781,26256.826172,26256.826172,8192867686,26270.425639,26679.198633,27087.971627,3.064357,-0.016634,27087.971627,26270.425639,26298.147754,2,0,48.079580,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
273,60609.496094,60780.500000,56555.292969,58254.011719,58254.011719,48439780271,59914.366438,63502.323828,67090.281218,11.300240,-0.231379,67090.281218,59914.366438,63878.574023,1,0,33.455914,2
287,61553.988281,66454.453125,61330.410156,66267.492188,66267.492188,39815167074,60003.842413,62349.128125,64694.413837,7.523075,1.335370,64694.413837,60003.842413,62322.341797,0,0,57.655745,1
288,66256.109375,66712.429688,64613.054688,65231.582031,65231.582031,31573077994,59886.867163,62556.091406,65225.315650,8.533859,1.001174,65225.315650,59886.867163,62396.154883,0,0,54.827566,1
289,65231.296875,67459.460938,65119.316406,67051.875000,67051.875000,28031279310,59687.016248,63027.797266,66368.578283,10.600977,1.102266,66368.578283,59687.016248,62577.791602,0,0,58.664557,1


In [13]:
data['entry'] = 0

# Condition for entry category 2 (buy entry)
buy_entry_condition = (data['Trend'] == 2) & (data['RSI Signal'] == 2) & (data['Low'] < data['Lower Band'])
data.loc[buy_entry_condition, 'entry'] = 2

# Condition for entry category 1 (sell entry)
sell_entry_condition = (data['Trend'] == 1) & (data['RSI Signal'] == 1) & (data['High'] > data['Upper Band'])
data.loc[sell_entry_condition, 'entry'] = 1

In [14]:
data[data['entry']!=0]

,Open,High,Low,Close,Adj Close,Volume,BBL_10_1.5,BBM_10_1.5,BBU_10_1.5,BBB_10_1.5,BBP_10_1.5,Upper Band,Lower Band,SMA,Trend,entry,RSI,RSI Signal
53,26579.373047,26716.058594,26221.050781,26256.826172,26256.826172,8192867686,26270.425639,26679.198633,27087.971627,3.064357,-0.016634,27087.971627,26270.425639,26298.147754,2,2,48.079580,2
70,27392.076172,27474.115234,26561.099609,26873.320312,26873.320312,13648094333,27102.278649,27587.461523,28072.644398,3.517416,-0.235951,28072.644398,27102.278649,27152.189063,2,2,46.984015,2
180,42030.914062,43305.867188,41818.332031,43288.246094,43288.246094,20668476578,39397.182119,41183.593750,42970.005381,8.675356,1.089073,42970.005381,39397.182119,42180.707617,1,1,55.550253,1
227,69392.484375,70046.273438,64801.394531,65315.117188,65315.117188,46842198371,66067.799433,69554.842969,73041.886504,10.026745,-0.107926,73041.886504,66067.799433,65836.782227,2,2,54.745462,2
244,69705.023438,69708.382812,64586.593750,65446.972656,65446.972656,50705240709,66880.723037,69340.201953,71799.680869,7.093948,-0.291474,71799.680869,66880.723037,67930.776172,2,2,47.286223,2
264,64935.632812,67233.960938,64548.179688,66837.679688,66837.679688,28282686673,62070.372046,64219.005469,66367.638892,6.691581,1.109381,66367.638892,62070.372046,66569.537500,1,1,51.815627,1


## 3 - Entry based on a rejection candle next to Bollinger Bands

In [15]:
def identify_shooting_star(data):
    # Create a new column for shooting star
    data['shooting_star'] = data.apply(lambda row: 2 if (
        ( (min(row['Open'], row['Close']) - row['Low']) > (1.5 * abs(row['Close'] - row['Open']))) and 
        (row['High'] - max(row['Close'], row['Open'])) < (0.8 * abs(row['Close'] - row['Open'])) and 
        (abs(row['Open'] - row['Close']) > row['Open'] * 0.01)
    ) else 1 if (
        (row['High'] - max(row['Open'], row['Close'])) > (1.5 * abs(row['Open'] - row['Close'])) and 
        (min(row['Close'], row['Open']) - row['Low']) < (0.8 * abs(row['Open'] - row['Close'])) and 
        (abs(row['Open'] - row['Close']) > row['Open'] * 0.01)
    ) else 0, axis=1)

    return data

In [16]:
data = identify_shooting_star(data)

In [17]:
data[data['shooting_star']!=0]

,Open,High,Low,Close,Adj Close,Volume,BBL_10_1.5,BBM_10_1.5,BBU_10_1.5,BBB_10_1.5,BBP_10_1.5,Upper Band,Lower Band,SMA,Trend,entry,RSI,RSI Signal,shooting_star
83,33077.304688,35150.433594,32880.761719,33901.527344,33901.527344,44934999645,26715.412692,29772.603516,32829.794339,20.536940,1.175281,32829.794339,26715.412692,28566.068652,2,0,87.157593,1,1
114,37296.316406,38415.339844,37261.605469,37720.281250,37720.281250,22922957823,36040.290339,37033.729688,38027.169036,5.365052,0.845543,38027.169036,36040.290339,36588.126172,0,0,61.645029,0,1
225,73079.375000,73750.070312,68563.023438,71396.593750,71396.593750,59594605698,64865.113406,69073.754297,73282.395188,12.185934,0.775961,73282.395188,64865.113406,64266.054688,2,0,72.370918,0,2
226,71387.875000,72357.132812,65630.695312,69403.773438,69403.773438,78320453976,66351.337305,69634.011719,72916.686132,9.428365,0.464931,72916.686132,66351.337305,65157.688281,2,0,65.773091,0,2
271,63106.363281,64174.878906,61795.457031,63841.121094,63841.121094,26635912073,62859.399577,64605.345703,66351.291829,5.404959,0.281143,66351.291829,62859.399577,64966.455273,0,0,45.540150,0,2
274,58253.703125,59602.296875,56937.203125,59123.433594,59123.433594,32711813559,59077.716960,62730.899219,66384.081477,11.647154,0.006257,66384.081477,59077.716960,63474.952344,1,0,36.404701,0,2
278,64038.312500,65494.902344,62746.238281,63161.949219,63161.949219,28697928697,59232.646483,62236.218359,65239.790236,9.652167,0.654105,65239.790236,59232.646483,63333.758203,0,0,47.877021,0,1
